In [ ]:
google_key="X" #
openaikey="x"
import os
os.environ["GOOGLE_API_KEY"]=google_key
os.environ["OPENAI_API_KEY"]=openaikey
os.environ["HUGGINGFACEHUB_API_TOKEN"]="X"

In [ ]:
!pip install langchain_google_genai
!pip install PyPDF2
!pip install langchain
!pip install faiss-cpu #gpu
!pip install langchain_openai
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.document_loaders import PyPDFLoader

def get_pdf_text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader= PdfReader(pdf)
        for page in pdf_reader.pages:
            text+= page.extract_text()
    return  text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

def get_vector_store(text_chunks):
    embeddings = OpenAIEmbeddings( )
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    return vector_store

pdf_docs = ["/content/resnet.pdf","/content/unet.pdf"]

texts=get_pdf_text(pdf_docs)
chunks=get_text_chunks(texts)
vector_store = FAISS.from_texts(chunks, OpenAIEmbeddings())

Similarity search within the vector database

In [ ]:
query = "best resnet models"
docs = vector_store.similarity_search(query)
print(docs[0].page_content)

Gemini Pro

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import HuggingFaceHub
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             convert_system_message_to_human=True)

memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7}),
        memory=memory
    )
response = conversation_chain.invoke("what is resnet tell me about it")
response

{'question': 'what is resnet tell me about it',
 'chat_history': [HumanMessage(content='what is resnet tell me about it'),
  AIMessage(content='ResNet, which stands for Residual Neural Network, is a type of deep neural network that was introduced in 2015 by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun. ResNets are designed to address the problem of vanishing gradients, which can occur in deep neural networks and make it difficult to train them.\n\nResNets are constructed by stacking residual blocks, which are composed of multiple convolutional layers and a shortcut connection. The shortcut connection allows the gradients to flow directly from the input of the residual block to the output, which helps to prevent the gradients from vanishing.\n\nResNets have achieved state-of-the-art results on a variety of image classification tasks, including the ImageNet Large Scale Visual Recognition Challenge (ILSVRC). In 2015, a ResNet won the ILSVRC competition with a top-5 error rate of 

In [ ]:
response["answer"]

GPT-4 0125-preview

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import HuggingFaceHub
llm = ChatOpenAI(model_name="gpt-4-0125-preview")

memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7}),
        memory=memory
    )
response = conversation_chain.invoke("what is resnet tell me about it")
response

{'question': 'what is resnet tell me about it',
 'chat_history': [HumanMessage(content='what is resnet tell me about it'),
  AIMessage(content='ResNet, short for Residual Network, is a type of convolutional neural network (CNN) architecture designed to enable the training of much deeper networks. It was introduced by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun from Microsoft Research in the paper "Deep Residual Learning for Image Recognition". The key innovation of ResNet is its use of residual learning blocks that make it easier to train very deep networks.\n\n### Key Features and Innovations:\n\n1. **Residual Blocks**: The core idea of ResNet is introducing a so-called "shortcut connection" that skips one or more layers. Traditional neural networks try to directly learn the desired underlying mapping from inputs to outputs. In contrast, ResNet reformulates the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced function